In [2]:

# coding: utf-8
# In[31]:

import itertools
import pandas as pd 
import numpy as np
import statsmodels.api as sm 
import matplotlib.pyplot as plt
from statsmodels.tsa.seasonal import seasonal_decompose

plt.style.use('fivethirtyeight')

Miami = pd.read_csv('https://raw.githubusercontent.com/Github-cai/Modern-data-analysis_group-work/main/weather_forcasting.csv',
                      parse_dates=['Week_ending'], index_col='Week_ending') 

Miami = Miami.fillna(Miami.bfill()) 
Miami.index = pd.to_datetime(Miami.index)
print(Miami) 
Miami.plot(figsize=(12, 6))

decomposition = seasonal_decompose(Miami['Max_Tem'], freq=12) 
fig = plt.figure()   
fig = decomposition.plot()   
fig.set_size_inches(30, 10)



# Define the p, d and q parameters to take any value between 0 and 2 
p = d = q = range(0, 2)   

# Generate all different combinations of p, q and q triplets 
pdq = list(itertools.product(p, d, q)) 
print(pdq)  

# Generate all different combinations of seasonal p, q and q triplets
seasonal_pdq = [(x[0], x[1], x[2], 12) for x in list(itertools.product(p, d, q))] 
print(seasonal_pdq)


for param in pdq:     
    for param_seasonal in seasonal_pdq:         
        try: 
            model = sm.tsa.statespace.SARIMAX(Miami, order=param, seasonal_order=param_seasonal, enforce_stationarity=False, enforce_invertibility=False)             
            results = model.fit() 
            print('ARIMA{}x{} - AIC:{}'.format(param, param_seasonal, results.aic))      
        except:             
            continue


# ARIMA(1, 0, 1)x(1, 1, 1, 12) - AIC:1359.731162757304 has the smallest AIC
# Fit the best parameter to the SARIMAX model
model = sm.tsa.statespace.SARIMAX(Miami, order=(1, 0, 1), seasonal_order=(1, 1, 1, 12), enforce_stationarity=False, enforce_invertibility=False) 
results = model.fit() 
print(results.summary().tables[1])

# Model diagnosis and investigation of any abnormal behavior
results.plot_diagnostics(figsize=(12, 12)) 
plt.show()



# Prediction Validation

## one-step ahead forecasat
pred = results.get_prediction(start=pd.to_datetime('2017-01-07'), dynamic=False) 
pred_ci = pred.conf_int()

ax = Miami['2014':].plot(label='Observed',figsize=(12, 6))

pred.predicted_mean.plot(ax=ax, label='One-step ahead Forecast', alpha=.7) 

ax.fill_between(pred_ci.index,
                pred_ci.iloc[:, 0],
                pred_ci.iloc[:, 1], color='k', alpha=.2)   

ax.set_xlabel('Date') 
ax.set_ylabel('Maximum Temperature (F)') 
plt.legend()   
plt.show()




ModuleNotFoundError: No module named 'statsmodels'